# Fetching Data from CryptoCompare webpage for News Data for the 10 currencies

Returns latest news
For Sentiment Analysis

In [1]:
# import libraries
import requests
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
from cryptocompare_config import API_Key
import numpy as np
import pymongo
from pymongo import MongoClient
import certifi
from mongo2_config import mongo_uri
import datetime
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

In [2]:
api_endpoint = 'https://min-api.cryptocompare.com/data/v2/news/?lang=EN&api_key='
api_endpoint += API_Key

In [3]:
data_retrieved = requests.get(api_endpoint).json()
data_retrieved

{'Type': 100,
 'Message': 'News list successfully returned',
 'Promoted': [],
 'Data': [{'id': '27429837',
   'guid': 'https://chaintimes.com/?p=4473',
   'published_on': 1643282323,
   'imageurl': 'https://images.cryptocompare.com/news/default/chaintimes.png',
   'title': 'Russian President says crypto-mining could give them certain competitive advantages.',
   'url': 'https://chaintimes.com/russian-president-says-crypto-mining-could-give-them-certain-competitive-advantages/',
   'source': 'chaintimes',
   'body': 'The President of the Russian Federation – Vladimir Putin – agreed with the country’s central bank that cryptocurrencies are highly volatile and pose several risks to the financial network. However, he sees potential in cryptocurrency mining as it could give Russia “certain competitive advantages.” As reported last week, the Bank of Russia intensified its harsh [&#8230;]Read full post Russian President says crypto-mining could give them certain competitive advantages. on Cha

In [5]:
# symbols=['BTC','ETH','XRP','USDT','ADA','XMR','XLM','BUSD','LTC','DOGE']
# symbols={'BTH':'928962','BTC':'1182','ETH':'7605', 'XRP': '5031', 'USDT': '171986', 'ADA': '321992','XMR': '5038', 'XLM': '4614', 'BUSD' : '930595', 'LTC' : '3808', 'DOGE': '4432'}
Top_news_df = pd.DataFrame()
# for symbol,coin_id in symbols.items():
#     url = 'https://min-api.cryptocompare.com/data/social/coin/histo/day?coinId='
#     url = url + coin_id
#     params = '&aggregate=2&limit=2000&toTs=1643261780'
#     url = url + params
#     url = url + '&api_key='+ API_Key    
#     data_retrieved = requests.get(url).json()
data_retrieved_json = data_retrieved['Data']
news_df = pd.json_normalize(data_retrieved_json)
# news_df["Coin_Symbol"] = symbol
news_df["Rev_time"] = pd.to_datetime(news_df['published_on'],unit='s')
Top_news_df = Top_news_df.append(news_df, ignore_index = True)

Top_news_df

,id,guid,published_on,imageurl,title,url,source,body,tags,categories,upvotes,downvotes,lang,source_info.name,source_info.lang,source_info.img,Rev_time
0,27429837,https://chaintimes.com/?p=4473,1643282323,https://images.cryptocompare.com/news/default/...,Russian President says crypto-mining could giv...,https://chaintimes.com/russian-president-says-...,chaintimes,The President of the Russian Federation – Vlad...,Crypto Exchange|Crypto News|Cryptocurrency|Tre...,Regulation|Fiat|Mining,2,1,EN,Chaintimes,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:18:43
1,27429832,https://thenewscrypto.com/?p=24029,1643281800,https://images.cryptocompare.com/news/default/...,"Reddit Follows Twitters Footsteps, Plans NFT a...",https://thenewscrypto.com/reddit-follows-twitt...,thenewscrypto,Reddit hasn&#8217;t determined exactly how it ...,NFT News|NFT|Reddit,Other,0,0,EN,The News Crypto,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:10:00
2,27429834,https://decrypt.co/?p=91344,1643281670,https://images.cryptocompare.com/news/default/...,NFTs 'Achieved Mainstream Breakthrough' in 202...,https://decrypt.co/91344/nfts-achieved-mainstr...,decrypt,Research firm Nansen found that total sales vo...,NFTs|ETH,ETH,1,1,EN,Decrypt,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:07:50
3,27429833,https://cointelegraph.com/news/bitcoin-dips-8-...,1643281645,https://images.cryptocompare.com/news/default/...,Bitcoin dips 8% from highs as trader demands B...,https://cointelegraph.com/news/bitcoin-dips-8-...,cointelegraph,"Bitcoin ""isn't shouting for bullishness"" below...",Bitcoin|BTC price,BTC|Market,1,1,EN,CoinTelegraph,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:07:25
4,27429836,https://www.cryptoglobe.com/latest/2022/01/ada...,1643281476,https://images.cryptocompare.com/news/cryptogl...,$ADA: Charles Hoskinson Is ‘Really Looking For...,https://www.cryptoglobe.com/latest/2022/01/ada...,cryptoglobe,"On Wednesday (January 26), Charles Hoskinson, ...",Cardano|DEXes,ADA|Technology,1,0,EN,CryptoGlobe,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:04:36
5,27429830,https://thenewscrypto.com/?p=24028,1643281444,https://images.cryptocompare.com/news/default/...,What will social media ownership look like in ...,https://thenewscrypto.com/what-will-social-med...,thenewscrypto,"Over the past two decades, social media has gr...",Editors News|#Creaton|#NFT|#platform|#Web2,Other,0,0,EN,The News Crypto,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:04:04
6,27429829,https://news.bitcoin.com/?p=506692&preview=tru...,1643281252,https://images.cryptocompare.com/news/default/...,US Sentences Bitcoin Seller to 1 Year in Jail ...,https://news.bitcoin.com/us-sentences-bitcoin-...,bitcoin.com,A bitcoin seller has been sentenced to one yea...,Regulation|Bitcoin Fraud|bitcoin purchases|Bit...,BTC|Regulation|Business,0,0,EN,Bitcoin.com,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:00:52
7,27429826,https://dailycoin.com/?p=37221,1643281244,https://images.cryptocompare.com/news/default/...,The Sandbox and Brinc announce US$50M Open Met...,https://dailycoin.com/the-sandbox-and-brinc-an...,dailycoin,The Sandbox Metaverse Accelerator Program will...,Press Releases|Blockchain|Blockchain Gaming|me...,Blockchain|Sponsored,0,0,EN,Daily Coin,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:00:44
8,27429827,https://www.cryptoknowmics.com/news/?p=226422,1643281239,https://images.cryptocompare.com/news/default/...,MoonPay Adds Credit Card Support to Make NFT P...,https://www.cryptoknowmics.com/news/moonpay-ad...,cryptoknowmics,"MoonPay, the crypto unicorn driving NFT mania ...",Breaking News|Live Crypto News|News|NFTs|MoonP...,ICO|Blockchain|ETH|Market,0,0,EN,Cryptoknowmics,EN,https://images.cryptocompare.com/news/default/...,2022-01-27 11:00:39
9,27429828,https://blockworks.co/?p=16103,1643281200,https://images.cryptocompare.com/news/default

In [7]:
# Replace NaN as '0'
Top_news_df.isnull().sum().sum()
Top_news_df.isna().sum()

id                  0
guid                0
published_on        0
imageurl            0
title               0
url                 0
source              0
body                0
tags                0
categories          0
upvotes             0
downvotes           0
lang                0
source_info.name    0
source_info.lang    0
source_info.img     0
Rev_time            0
dtype: int64

In [6]:
# change column names
Top_stats_df= Top_stats_df.rename(columns={   
    
    'followers': 'Cryptocompare_followers', 
    'total_page_views': 'Cryptocompare_total_page_views', 
    'fb_talking_about': 'Fb_followers',
    'twitter_followers' : 'Twitter_followers',
    'reddit_subscribers': 'Reddit_subscribers',
    'code_repo_subscribers': 'Code_Repo_Subscribers'})
                                
Top_stats_df

,comments,posts,Cryptocompare_followers,points,overview_page_views,analysis_page_views,markets_page_views,charts_page_views,trades_page_views,forum_page_views,...,code_repo_forks,Code_Repo_Subscribers,code_repo_open_pull_issues,code_repo_closed_pull_issues,code_repo_open_issues,code_repo_closed_issues,code_repo_contributors,time,Coin_Symbol,Rev_time
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1470355200,BTH,2016-08-05
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1470528000,BTH,2016-08-07
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1470700800,BTH,2016-08-09
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1470873600,BTH,2016-08-11
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1471046400,BTH,2016-08-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11006,6504,4358,18291,631315,2118043,30987,72058,208922,67071,283150,...,2290,861,65,1390,94,902,593,1642464000,DOGE,2022-01-18
11007,6504,4358,18303,631615,2118715,30996,72072,208965,67087,283191,...,2290,861,65,1390,94,902,593,1642636800,DOGE,2022-01-20
11008,6505,4359,18306,631725,2119577,31006,72083,209027,67104,283255,...,2290,861,65,1390,94,902,593,1642809600,DOGE,2022-01-22
11009,6505,4359,18312,631875,2120388,31020,72098,209091,67152,283311,...,2290,861,65,1390,94,902,593,1642982400,DOGE,2022-01-24


In [7]:
Top_stats_df.dtypes

comments                                   int64
posts                                      int64
Cryptocompare_followers                    int64
points                                     int64
overview_page_views                        int64
analysis_page_views                        int64
markets_page_views                         int64
charts_page_views                          int64
trades_page_views                          int64
forum_page_views                           int64
influence_page_views                       int64
Cryptocompare_total_page_views             int64
fb_likes                                   int64
Fb_followers                               int64
Twitter_followers                          int64
twitter_following                          int64
twitter_lists                              int64
twitter_favourites                         int64
twitter_statuses                           int64
Reddit_subscribers                         int64
reddit_active_users 

In [14]:
#creating new df for only the number of followers from Top_news_df
Followers_df = Top_stats_df[['Coin_Symbol','Rev_time','Cryptocompare_followers', 'Cryptocompare_total_page_views', 'Fb_followers', 'Twitter_followers', 'Reddit_subscribers', 'Code_Repo_Subscribers']]

Followers_df    

,Coin_Symbol,Rev_time,Cryptocompare_followers,Cryptocompare_total_page_views,Fb_followers,Twitter_followers,Reddit_subscribers,Code_Repo_Subscribers
0,BTH,2016-08-05,0,0,0,0,0,0
1,BTH,2016-08-07,0,0,0,0,0,0
2,BTH,2016-08-09,0,0,0,0,0,0
3,BTH,2016-08-11,0,0,0,0,0,0
4,BTH,2016-08-13,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
11006,DOGE,2022-01-18,18291,2786137,63,2887256,2260882,861
11007,DOGE,2022-01-20,18303,2786934,63,2902233,2261906,861
11008,DOGE,2022-01-22,18306,2787960,63,2912560,2262813,861
11009,DOGE,2022-01-24,18312,2788970,63,2922496,2263703,861


In [9]:
# adding a new column with total number of followers and subscribers
# Top_stats_df.insert(6, "Total_Follower_Subscriber", sum, True)

col_list = list(Followers_df)
print(col_list)

Total_Followers_Subscribers = Followers_df[col_list].sum(axis=1)
Followers_df["Total_Followers_Subscribers"] = Total_Followers_Subscribers
Followers_df

['Coin_Symbol', 'Rev_time', 'Cryptocompare_followers', 'Cryptocompare_total_page_views', 'Fb_followers', 'Twitter_followers', 'Reddit_subscribers', 'Code_Repo_Subscribers']


C:\Users\lavin\AppData\Local\Temp/ipykernel_9608/2330592547.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Total_Followers_Subscribers = Followers_df[col_list].sum(axis=1)
C:\Users\lavin\AppData\Local\Temp/ipykernel_9608/2330592547.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Followers_df["Total_Followers_Subscribers"] = Total_Followers_Subscribers


,Coin_Symbol,Rev_time,Cryptocompare_followers,Cryptocompare_total_page_views,Fb_followers,Twitter_followers,Reddit_subscribers,Code_Repo_Subscribers,Total_Followers_Subscribers
0,BTH,2016-08-05,0,0,0,0,0,0,0
1,BTH,2016-08-07,0,0,0,0,0,0,0
2,BTH,2016-08-09,0,0,0,0,0,0,0
3,BTH,2016-08-11,0,0,0,0,0,0,0
4,BTH,2016-08-13,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
11006,DOGE,2022-01-18,18291,2786137,63,2887256,2260882,861,7953490
11007,DOGE,2022-01-20,18303,2786934,63,2902233,2261906,861,7970300
11008,DOGE,2022-01-22,18306,2787960,63,2912560,2262813,861,7982563
11009,DOGE,2022-01-24,18312,2788970,63,2922496,2263703,861,7994405


# Exporting data to csv and Uploading on MongoDb

In [15]:
# exporting the dataframe to csv
Followers_df.to_csv('Social_media_follower_count(5year).csv')

In [16]:
print(Followers_df.dtypes)

Coin_Symbol                               object
Rev_time                          datetime64[ns]
Cryptocompare_followers                    int64
Cryptocompare_total_page_views             int64
Fb_followers                               int64
Twitter_followers                          int64
Reddit_subscribers                         int64
Code_Repo_Subscribers                      int64
dtype: object


In [13]:
# Followers_df = Followers_df.applymap(str)

In [17]:
print(Followers_df.dtypes)

Coin_Symbol                               object
Rev_time                          datetime64[ns]
Cryptocompare_followers                    int64
Cryptocompare_total_page_views             int64
Fb_followers                               int64
Twitter_followers                          int64
Reddit_subscribers                         int64
Code_Repo_Subscribers                      int64
dtype: object


In [18]:
# Read in mongodb server location as client
client = MongoClient(mongo_uri,tlsCAFile=certifi.where())

In [19]:
# Find Databases
client.list_database_names()

['Kaggle_data', 'Latest_Cryptodata', 'Latest_StockMarket', 'admin', 'local']

In [20]:
# connecting to the db on MongoDb and creating collection in the db
db = client['Latest_Cryptodata']
collection = db['Social_stats_daily_data']

In [21]:
Followers_df.reset_index(inplace=True)
Followers_df_dict = Followers_df.to_dict("records")

In [22]:
# Insert collection
collection.insert_many(Followers_df_dict)

# Create visualizations using matplotlib
